In [1]:
!pip install simpy

In [0]:
import random
import math
import simpy

Una empresa de textiles tiene una maquina de cortes de tela que se demora entre 15 y 30 minutos por corte. La maquina recibe en promedio 3 rollos de tela por hora (es decir, uno rollo cada 20 minutos). Se desea simular las llegadas y servicios de 5 rollos de tela. 

In [0]:
SEMILLA = 30
MAQUINA_DE_CORTE = 1
TIEMPO_CORTE_MIN = 15
TIEMPO_CORTE_MAX = 30
T_LLEGADAS = 20
TIEMPO_SIMULACION = 120
TOT_ROLLO_TELA = 5

te  = 0.0 # tiempo de espera total
dt  = 0.0 # duracion de servicio total
fin = 0.0 # minuto en el que finaliza


In [0]:
def cortar(rollotela):
	global dt  #Para poder acceder a la variable dt declarada anteriormente
	R = random.random()  # Obtiene un numero aleatorio y lo guarda en R
	tiempo = TIEMPO_CORTE_MAX - TIEMPO_CORTE_MIN  
	tiempo_corte = TIEMPO_CORTE_MIN + (tiempo*R) # Distribucion uniforme
	yield env.timeout(tiempo_corte) # deja correr el tiempo n minutos
	print(" \o/ Corte listo a %s en %.2f minutos" % (rollotela,tiempo_corte))
	dt = dt + tiempo_corte # Acumula los tiempos de uso de la i
 
def tela (env, name, maquinas ):
	global te
	global fin
	llega = env.now # Guarda el minuto de llegada del rollo de tela
	print ("---> %s llego a la maquina en minuto %.2f" % (name, llega))
	with maquinas.request() as request: # Espera su turno
		yield request # Obtiene turno
		pasa = env.now # Guarda el minuto cuado comienza a ser cortada
		espera = pasa - llega # Calcula el tiempo que espero
		te = te + espera # Acumula los tiempos de espera
		print ("**** %s pasa a la maquina en minuto %.2f habiendo esperado %.2f" % (name, pasa, espera))
		yield env.process(cortar(name)) # Invoca al proceso cortar
		deja = env.now #Guarda el minuto en que termina el proceso cortar 
		print ("<--- %s sale de la maquina en minuto %.2f" % (name, deja))
		fin = deja # Conserva globalmente el ultimo minuto de la simulacion
	
 
def principal (env, maquinas):
	inicio = 0
	i = 0
	for i in range(TOT_ROLLO_TELA): # Para n rollos de tela
		R = random.random()
		inicio = -T_LLEGADAS * math.log(R) # Distribucion exponencial
		yield env.timeout(inicio)  # Deja transcurrir un tiempo entre uno y otro
		i += 1
		env.process(tela(env, 'Rollo de tela %d' % i, maquinas)) 

In [15]:
print ("------------------- Simulacion de Maquina de Corte de Tela------------------")
random.seed (SEMILLA)  # Cualquier valor
env = simpy.Environment() # Crea el objeto entorno de simulacion
maquinas = simpy.Resource(env, TOT_ROLLO_TELA) #Crea los recursos (maquinas)
env.process(principal(env, maquinas)) #Invoca el proceso princial
env.run() #Inicia la simulacion


------------------- Simulacion de Maquina de Corte de Tela------------------
---> Rollo de tela 1 llego a la maquina en minuto 12.36
**** Rollo de tela 1 pasa a la maquina en minuto 12.36 habiendo esperado 0.00
 \o/ Corte listo a Rollo de tela 1 en 15.45 minutos
<--- Rollo de tela 1 sale de la maquina en minuto 27.81
---> Rollo de tela 2 llego a la maquina en minuto 37.17
**** Rollo de tela 2 pasa a la maquina en minuto 37.17 habiendo esperado 0.00
---> Rollo de tela 3 llego a la maquina en minuto 45.67
**** Rollo de tela 3 pasa a la maquina en minuto 45.67 habiendo esperado 0.00
 \o/ Corte listo a Rollo de tela 2 en 18.15 minutos
<--- Rollo de tela 2 sale de la maquina en minuto 55.32
 \o/ Corte listo a Rollo de tela 3 en 20.96 minutos
<--- Rollo de tela 3 sale de la maquina en minuto 66.63
---> Rollo de tela 4 llego a la maquina en minuto 72.83
**** Rollo de tela 4 pasa a la maquina en minuto 72.83 habiendo esperado 0.00
---> Rollo de tela 5 llego a la maquina en minuto 81.70
**** Ro

In [12]:
print ("\n---------------------------------------------------------------------")
print ("\nIndicadores obtenidos: ")

lpc = te / fin
print ("\nLongitud promedio de la cola: %.2f" % lpc)
tep = te / TOT_ROLLO_TELA
print ("Tiempo de espera promedio = %.2f" % tep)
upi = (dt / fin) / MAQUINA_DE_CORTE
print ("Uso promedio de la instalacion = %.2f" % upi)
print ("\n---------------------------------------------------------------------")


---------------------------------------------------------------------

Indicadores obtenidos: 

Longitud promedio de la cola: 0.00
Tiempo de espera promedio = 0.00
Uso promedio de la instalacion = 1.03

---------------------------------------------------------------------
